In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
%matplotlib qt

try:
    from tkinter import Tk
    from tkFileDialog import askopenfilenames
except:
    from tkinter import Tk
    from tkinter import filedialog

In [13]:
def gaussian(x, N, mu, sigma):
    return N*np.exp(-0.5*((x-mu)/sigma)**2)

def doubleGaussian(x, N1, mu1, sigma1, N2, mu2, sigma2):
    return N1*np.exp(-0.5*((x-mu1)/sigma1)**2) + N2*np.exp(-0.5*((x-mu2)/sigma2)**2) 

def evaluate_fwhm(x, y, threshold = 0.05):
    y_norm = y / np.max(y)

    try:
        i_min = np.where(y_norm >= 0.5*(1.-threshold))[0][0]
        i_max = np.where(y_norm >= 0.5*(1.-threshold))[0][-1] 
        if i_min == i_max:
            i_min -= 1
            i_max += 1
        x_min = x[i_min]
        x_max = x[i_max]
            
        fwhm = x_max - x_min
    except:
        fwhm = np.inf

    return fwhm, np.nan


### Set-up

In [49]:
use_calibrated = False #default = True
fit_type = 'dgauss' #Use 'dgauss' to fit a double gaussian (default), 'gauss' to fit a normal gaussian

#### Calibration

In [4]:
root = Tk()
root.withdraw() 
root.wm_attributes('-topmost', 1) 
calib_file_name = filedialog.askopenfilenames(parent = root) 

In [5]:

chanel_cal, energy_cal = np.loadtxt(calib_file_name[0], unpack = 'True')

#NB: Ordered from n-th order to 0-th order
calib_coeff = np.polyfit(chanel_cal, energy_cal, deg = 1)

In [6]:
mock_channel = np.linspace(0,2000,2000)
cal_line = mock_channel*calib_coeff[0] + calib_coeff[1]

fig = plt.figure()
ax = fig.add_subplot()

ax.set_title('Calibration Plot')
ax.plot(mock_channel, cal_line, color = 'grey', label = 'Cal. Line')
ax.scatter(chanel_cal, energy_cal, marker = '+', color ='red', label = 'Cal. Points')
ax.set_xlabel('Channel')
ax.set_ylabel('Energy [keV]')
ax.legend()
ax.grid()


#### Data Analysis

In [34]:
root = Tk()
root.withdraw() 
root.wm_attributes('-topmost', 1) 
data_file_name = filedialog.askopenfilenames(parent = root) 

In [35]:
chanel, counts = np.loadtxt(data_file_name[0], unpack = True, skiprows=10, delimiter =';', usecols=(0,1))
if (use_calibrated):
    calib_energies =  chanel*calib_coeff[0] + calib_coeff[1]


In [51]:
if use_calibrated:
    fig, ax = plt.subplots(1,2, figsize = (15,7))
    plt.suptitle(data_file_name[0])

    ax[0].step(chanel, counts, color = 'gray')
    ax[0].set_xlabel('Channel')
    ax[0].set_ylabel('Counts')
    ax[0].grid()

    ax[1].step(calib_energies, counts, color = 'gray')
    ax[1].set_xlabel('Energy [keV]')
    ax[1].set_ylabel('Counts')
    ax[1].grid()
else:
    fig, ax = plt.subplots(1,1, figsize = (7,7))
    plt.suptitle(data_file_name[0])

    ax.step(chanel, counts, color = 'gray')
    ax.set_xlabel('Channel')
    ax.set_ylabel('Counts')
    ax.grid()


In [54]:
#CAREFUL! If use_calibrated = True, those two values are energies, otherwhise they represent channel numbers
min_val = 80
max_val = 120

if use_calibrated:
      mask = np.logical_and(calib_energies >= min_val, calib_energies <= max_val)
      energies_to_fit = calib_energies[mask]
      x = energies_to_fit
else:
      mask = np.logical_and(chanel >= min_val, chanel <= max_val)
      chanel_to_fit = chanel[mask]
      x = chanel_to_fit

counts_to_fit = counts[mask]

x_fit = np.linspace(np.min(x), np.max(x),1000)

if fit_type == 'dgauss': 

      p0 = [np.max(counts_to_fit), np.min(x), 1, 
            np.max(counts_to_fit), np.min(x), 1]
      bounds = [(0,min(x),0,0,min(x),0),
            (np.inf, max(x),np.inf, np.inf, max(x), np.inf)]

      
      par, cov = curve_fit(doubleGaussian, xdata = x, ydata = counts_to_fit, p0 = p0, bounds = bounds)      
      y_fit = doubleGaussian(x_fit, *par)

      #fwhm, err_fwhm = evaluate_fwhm(x_fit, y_fit)
      fwhm = par[5]*2.355
      #Must be corrected to take into account covariance
      errs = np.sqrt(np.diag(cov))
      err_fwhm = errs[5]*2.355

elif fit_type == 'gauss':
      p0 = [np.max(x), np.min(x), 1]
      bounds = [(0,min(x),0), (np.inf, max(x),np.inf)]

      par, cov = curve_fit(gaussian, xdata = x, ydata = counts_to_fit, p0 = p0, bounds = bounds)
      y_fit = gaussian(x_fit, *par)

      fwhm = par[2]*2.355
      #Must be corrected to take into account covariance
      errs = np.sqrt(np.diag(cov))
      err_fwhm = errs[2]*2.355

else:
      raise Exception("Select a valid fit function: either double gaussian (\'dgauss\'), or gaussian (\'gauss\')")

fig, ax = plt.subplots(1,1, figsize = (9,7))
ax.step(x, counts_to_fit, color = 'black', label = 'Data')
if use_calibrated:
      ax.set_xlabel('Energy [keV]')
      um = 'keV'
else:
      ax.set_xlabel('Channel')
      um = 'ch'
ax.set_ylabel('Counts')
ax.grid()

ax.plot(x_fit, y_fit, color = 'red', label = 'Fit')
ax.legend()

print('Data File: ', data_file_name[0])
print('Calibration: ', calib_file_name[0])
print('\n')
print('Fitting range: {0} - {1} {2}'.format(min_val, max_val, um))
print('Fitting model: double gaussian')
print('\n')
print('FIT PARAMETERS:')
print('N1:     {0:.2f} ± {1:.2f} counts'.format(par[0],errs[0]))
print('Mu1:    {0:.2f} ± {1:.2f} {2}'.format(par[1],errs[1], um))
print('Sigma1: {0:.2f} ± {1:.2f} {2}'.format(par[2],errs[2], um))
if fit_type == 'dgauss':
      print('N2:     {0:.2f} ± {1:.2f} counts'.format(par[3],errs[3]))
      print('Mu2:    {0:.2f} ± {1:.2f} {2}'.format(par[4],errs[4], um))
      print('Sigma2: {0:.2f} ± {1:.2f} {2}'.format(par[5],errs[5], um))
print('\n')

print('FWHM (main peak): {0:.2f} ± {1:.2f} {2}'.format(fwhm, err_fwhm, um))


Data File:  C:/Users/lisaf/Desktop/due2lab/test_Am
Calibration:  C:/Users/lisaf/Desktop/due2lab/calib.txt


Fitting range: 80 - 120 ch
Fitting model: double gaussian


FIT PARAMETERS:
N1:     300.40 ± 27.00 counts
Mu1:    94.05 ± 1.41 ch
Sigma1: 12.08 ± 0.65 ch
N2:     1521.90 ± 37.65 counts
Mu2:    103.91 ± 0.07 ch
Sigma2: 5.57 ± 0.10 ch


FWHM (main peak): 13.11 ± 0.24 ch
